1. 背景描述：一个农民承包了6块耕地共300亩，准备播种小麦，玉米，水果和蔬菜四种农产品，已知各种农产品的计划播种面积、每块土地种植不同农产品的单产收益

2. 任务描述：如何进行合理安排，使得总收益最大

||地块1|地块2|地块3|地块4|地块5|地块6|计划播种面积（亩）|
--|--|--|--|--|--|--|--
小麦|500|550|630|1000|800|700|76
玉米|800|700|600|950|900|930|88
水果|1000|960|840|650|600|700|96
蔬菜|1200|1040|980|860|880|780|40
面积（亩）|42|56|44|39|60|59|\

In [1]:
import pandas as pd
import numpy as np
import pulp

In [2]:
# 目标函数系数
z = np.array([
    [500,550,630,1000,800,700],
    [800,700,600,950,900,930],
    [1000,960,840,650,600,700],
    [1200,1040,980,860,880,780]
])

# 约束系数
a = np.ones(z.shape)

# 约束值
b1 = np.array([76, 88, 96, 40])
b2 = np.array([42, 56, 44, 39, 60, 59])

In [3]:
# 定义最大化问题
problem = pulp.LpProblem(name='farmland', sense=pulp.LpMaximize)

# 定义变量函数
x = [pulp.LpVariable('x{}'.format(i), lowBound=0) for i in range(z.size)]

# 设置目标函数
problem += pulp.lpDot(z.ravel(), x)

# 计划播种面积约束（大于等于）
for i in range(1, len(b1)+1):
    problem += (pulp.lpDot(a[i-1], x[(i*6)-6:(i*6)]) >= b1[i-1])

# 地块面积约束（小于等于）
for i in range(0, len(b2)):
    problem += (pulp.lpDot(a[:,i], x[i::6]) <= b2[i])

# 执行求解
problem.solve()

# 结果输出
data = np.array([pulp.value(i) for i in x],dtype='int').reshape(4, 6)
df_result = pd.DataFrame(data=data, index=['小麦','玉米','水果','蔬菜'], columns=['地块1','地块2','地块3','地块4','地块5','地块6'])
df_result

,地块1,地块2,地块3,地块4,地块5,地块6
小麦,0,0,6,39,31,0
玉米,0,0,0,0,29,59
水果,2,56,38,0,0,0
蔬菜,40,0,0,0,0,0


In [5]:
print('最大化收入', pulp.value(problem.objective))
print('是否符合播种面积', (df_result.sum(axis=1).values <= b1).any())
print('是否符合地块面积', (df_result.sum(axis=0).values <= b2).any())

最大化收入 284230.0
是否符合播种面积 True
是否符合地块面积 True
